![HenryLogo](https://d31uz8lwfmyn8g.cloudfront.net/Assets/logo-henry-white-lg.png)

# **Proyecto integrador 2 - modelo de clasificación**

### **Planteamiento de la problemática**
​
Hemos sido contratados en el equipo de ciencias de datos en una consultora de renombre. Nos han asignado a un proyecto de estudio de mercado de una importante automotriz china. Nuestro cliente desea ingresar a nuestro mercado de automóviles, por lo que nos han encomendado analizar las características de los vehículos presentes en el mercado actual. Dado que tienen en su catálogo una amplia colección de modelos de todo tipo, cuyo catálogo está estratificado en gamas según el gusto de cada región, desean saber qué características presentan los vehículos de gama alta y los de gama baja en nuestro mercado, para poder abarcar todo los públicos objetivos ajustándose a toda la demanda y, en base a estos datos, poder cotizar correctamente los vehículos que ofrecerá. 

Para ello, nuestro departamento de datos ha recopilado precios y características de varios de los modelos de vehículos disponibles en nuestro mercado, junto con sus precios de venta al público.


===========================================================

Éstas son las dos predicciones que se analizarán: ​ 

1 Implementar un modelo de clasificación con aprendizaje supervisado que permita clasificar el precio de los vehículos en baratos y caros usando la mediana de los precios como punto de corte, utilizando los datos que se han puesto a su disposición. 

2 Implementar un modelo de regresión con aprendizaje supervisado que permita predecir el precio final de los vehículos, utilizando los datos que se han puesto a su disposición.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


## Carga y análisis

In [ ]:
df = pd.read_csv(r"E:\Repositorios y bases de datos\Henry DS\Módulo 6\Proyecto_Integrador\Propuesta 2\ML_cars.csv")

In [ ]:
df.isnull()

In [ ]:
# Visualización de distribuciones y relaciones
plt.figure(figsize=(10, 6))
sns.histplot(df['price'], bins=20, kde=True)  
plt.title('Distribución de Precios de Vehículos')
plt.xlabel('Precio')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Visualización de las relaciones entre variables
plt.figure(figsize=(10, 8))
sns.scatterplot(x='enginesize', y='horsepower', data=df, hue='fueltype')
plt.title('Relación entre Tamaño del Motor y Potencia por Tipo de Combustible')
plt.xlabel('Tamaño del Motor')
plt.ylabel('Potencia')
plt.show()

## Transformaciones

In [ ]:
columnas_dummies = ['enginetype', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginelocation', 'fuelsystem']
datos = pd.get_dummies(df, columns=columnas_dummies, drop_first=True)

datos

In [ ]:
datos = datos.drop(columns=['car_ID', 'CarName'])

In [ ]:
# Alta o baja según la mediana
median_price = datos['price'].median()
datos['gama'] = datos['price'].apply(lambda x: 'Alta' if x > median_price else 'Baja')


In [ ]:
print(datos.columns)

## 1) MODELO DE CLASIFICACIÓN

In [ ]:
# Definición de la mediana de los precios
median_price = df['price'].median()

datos['price_category'] = np.where(datos['price'] <= median_price, 'barato', 'caro')

# Configuración de train
X_train = datos.drop('price_category', axis=1)  
y_train = datos['price_category']


print(y_train.unique()) 

In [ ]:
X_train_encoded = pd.get_dummies(X_train)

# Entrenamiento
model = RandomForestClassifier(random_state=42)

# SE PROCEDE A REALIZAR VALIDACIÓN CRUZADA YA QUE LOS RESULTADOS OBTENIDOS SUGERÍAN OVERFITTING
scores = cross_val_score(model, X_train_encoded, y_train, cv=5)

print("Puntajes de precisión por pliegue:", scores)
print("Precisión media:", scores.mean())